In [2]:
from rdkit import Chem
from functions import bfs_traversal, find_n_terminus
from rdkit.Chem import AllChem
import numpy as np


In [58]:
smiles = "CC(C)[C@H](C(N[C@H](CCC(OCC=C)=O)C(NC(C)(C)C(N(c(cc(cc1)C(NC)=O)c1N1C)C1=O)=O)=O)=O)N(C)C([C@H](Cc1c[nH]c2c1cccc2)NC(CN(C)C(CCN)=O)=O)=O"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
print(mol.GetNumAtoms())

xyz_file = "/Users/zaan/zasaeed@g.hmc.edu - Google Drive/My Drive/OMO Lab - Peptide Cyclization - Zaan Saeed/Data/NewPeptideLibrary/Peptide_R1C1/R1C1_Conformations/R1C1_Conformation_1.xyz"

def load_xyz_coords(mol, xyz_path):
    conf = Chem.Conformer(mol.GetNumAtoms())

    with open(xyz_path, 'r') as f:
        lines = f.readlines()
        lines = lines[2:]

        for i, line in enumerate(lines):
            tokens = line.strip().split()
            if len(tokens) < 4:
                continue
            x, y, z = map(float, tokens[1:4])
            conf.SetAtomPosition(i, (x, y, z))

    mol.RemoveAllConformers()
    mol.AddConformer(conf, assignId=True)
    return mol

mol = load_xyz_coords(mol, xyz_file)



n_terminus = find_n_terminus(mol)
nitrogen_order = bfs_traversal(mol, n_terminus)
###########

n_terminus_residue_normal = mol.GetSubstructMatches(Chem.MolFromSmarts('[NH2]C[C](=O)[N]'))
n_terminus_residue_abnormal = mol.GetSubstructMatches(Chem.MolFromSmarts('[NH2]CC[C](=O)[N]'))



normal_residues = mol.GetSubstructMatches(Chem.MolFromSmiles('C(=O)NCC(=O)N'))
abnormal_residues = mol.GetSubstructMatches(Chem.MolFromSmiles('C(=O)NCCC(=O)N'))

all_residues = normal_residues + abnormal_residues +n_terminus_residue_normal + n_terminus_residue_abnormal
all_residues = list(all_residues)
############## get rid of asparagine

query1 = Chem.MolFromSmarts('C(=O)[N]C[C](=O)[NH2]')
query2= Chem.MolFromSmarts('C(=O)[N]CC[C](=O)[NH2]')
asparagines = mol.GetSubstructMatches(query1)+mol.GetSubstructMatches(query2)


for asparagine in asparagines:
    for residue in all_residues:
        if asparagine[-1] in residue:
            all_residues.remove(residue)
##############


ordered_residues = []

for id in nitrogen_order:
    for residue in all_residues:
        if id in residue:
            ordered_residues.append(residue)
            all_residues.remove(residue)
print(ordered_residues)
    # convert xyz to mol object, iterate thru the atoms figuring out which ones to calculate dihederal of, and then get coordinates #thru mol commands (.getposition) or sort

125
[(61, 60, 59, 58, 62, 56), (58, 62, 56, 55, 54, 63, 53), (54, 63, 53, 42, 41, 64, 39), (41, 64, 39, 3, 4, 38, 5), (4, 38, 5, 6, 15, 37, 16), (15, 37, 16, 17, 20, 36, 21)]


In [59]:
def calculate_dihedrals(residue):
    def get_dihedral_angle(p1, p2, p3, p4):
        """Calculate the dihedral angle between four points in 3D space."""
        # Convert to numpy arrays
        p1, p2, p3, p4 = map(np.array, (p1, p2, p3, p4))

        # Bond vectors
        b1 = p2 - p1
        b2 = p3 - p2
        b3 = p4 - p3

        # Normal vectors
        n1 = np.cross(b1, b2)
        n2 = np.cross(b2, b3)

        # Normalize normals
        n1 /= np.linalg.norm(n1)
        n2 /= np.linalg.norm(n2)

        # Normalize b2
        b2_unit = b2 / np.linalg.norm(b2)

        # Compute the angle
        x = np.dot(n1, n2)
        y = np.dot(np.cross(n1, n2), b2_unit)

        angle = np.arctan2(y, x)
        return np.degrees(angle)



    if len(residue) == 5: #n-terminus case (normal) (5000,5000,psi) [NH2]C[C](=O)[N]
        temp_dihedrals = []
        temp_dihedrals.append(5000)
        temp_dihedrals.append(5000)
        p1= mol.GetConformer().GetAtomPosition(residue[0])
        p2 = mol.GetConformer().GetAtomPosition(residue[1])
        p3= mol.GetConformer().GetAtomPosition(residue[2])
        p4 = mol.GetConformer().GetAtomPosition(residue[4])
        temp_dihedrals.append(get_dihedral_angle(p1,p2,p3,p4))
        return temp_dihedrals

    if len(residue) == 6: #n-terminus case (abnormal) (5000,theta,psi) [NH2]CC[C](=O)[N]
        temp_dihedrals = []
        temp_dihedrals.append(5000)
        p1= mol.GetConformer().GetAtomPosition(residue[0])
        p2= mol.GetConformer().GetAtomPosition(residue[1])
        p3=mol.GetConformer().GetAtomPosition(residue[2])
        p4 = mol.GetConformer().GetAtomPosition(residue[3])
        temp_dihedrals.append(get_dihedral_angle(p1,p2,p3,p4))
        p1= mol.GetConformer().GetAtomPosition(residue[1])
        p2= mol.GetConformer().GetAtomPosition(residue[2])
        p3=mol.GetConformer().GetAtomPosition(residue[3])
        p4 = mol.GetConformer().GetAtomPosition(residue[5])

        temp_dihedrals.append(get_dihedral_angle(p1,p2,p3,p4))
        return temp_dihedrals
    if len(residue) == 7: #normal   (phi,5000,psi) C(=O)NCC(=O)N
        temp_dihedrals = []
        p1= mol.GetConformer().GetAtomPosition(residue[0])
        p2 = mol.GetConformer().GetAtomPosition(residue[2])
        p3 = mol.GetConformer().GetAtomPosition(residue[3])
        p4 = mol.GetConformer().GetAtomPosition(residue[4])
        temp_dihedrals.append(get_dihedral_angle(p1,p2,p3,p4))
        temp_dihedrals.append(5000)
        p1= mol.GetConformer().GetAtomPosition(residue[2])
        p2 = mol.GetConformer().GetAtomPosition(residue[3])
        p3 = mol.GetConformer().GetAtomPosition(residue[4])
        p4 = mol.GetConformer().GetAtomPosition(residue[6])
        temp_dihedrals.append(get_dihedral_angle(p1,p2,p3,p4))
        return temp_dihedrals

    if len(residue)==8: #abnormal case (phi,theta,psi) C(=O)NCCC(=O)N
        temp_dihedrals = []
        p1= mol.GetConformer().GetAtomPosition(residue[0])
        p2 = mol.GetConformer().GetAtomPosition(residue[2])
        p3 = mol.GetConformer().GetAtomPosition(residue[3])
        p4 = mol.GetConformer().GetAtomPosition(residue[4])
        temp_dihedrals.append(get_dihedral_angle(p1,p2,p3,p4))
        p1= mol.GetConformer().GetAtomPosition(residue[2])
        p2 = mol.GetConformer().GetAtomPosition(residue[3])
        p3 = mol.GetConformer().GetAtomPosition(residue[4])
        p4 = mol.GetConformer().GetAtomPosition(residue[5])
        temp_dihedrals.append(get_dihedral_angle(p1,p2,p3,p4))
        p1= mol.GetConformer().GetAtomPosition(residue[3])
        p2 = mol.GetConformer().GetAtomPosition(residue[4])
        p3 = mol.GetConformer().GetAtomPosition(residue[5])
        p4 = mol.GetConformer().GetAtomPosition(residue[7])
        temp_dihedrals.append(get_dihedral_angle(p1,p2,p3,p4))
        return temp_dihedrals







In [60]:
dihedrals = []
for residue in ordered_residues:
    dihedrals.append(calculate_dihedrals(residue))
print(dihedrals)
len(dihedrals)
for d in dihedrals:
    print(len(d))

[[5000, np.float64(-177.45900344010505), np.float64(108.2113555883941)], [np.float64(-77.69454543958622), 5000, np.float64(-17.539564460285558)], [np.float64(-150.27934392974552), 5000, np.float64(154.51728140221815)], [np.float64(-64.90787024394047), 5000, np.float64(105.50862493620524)], [np.float64(135.37661903451843), 5000, np.float64(-16.598705858775052)], [np.float64(-158.0124423693613), 5000, np.float64(78.34102446328443)]]
3
3
3
3
3
3
